<a href="https://colab.research.google.com/github/gorbo96/Simulacion/blob/main/ExamenSimulacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install simpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import collections
import random
import simpy
import matplotlib.pyplot as plt
import math
import numpy as np 
from scipy import stats

In [4]:
class Simulacion():
  CAPACIDAD_BUS=40
  TIEMPO_SIMULACION = 168
  LUGARES=["Quito","Cuenca","Machala","Guayaquil","Loja","Zamora"]
  PROB_LUGARES=[0.16,0.08,0.16,0.08,0.32,0.16]
  VELOCIDAD_LLEGADA = 0.3
  Cliente= collections.namedtuple('Cliente', 'genero,edad,origen,destino,boletos')
  Empresa= collections.namedtuple('Empresa', 'bus1,bus2,bus3,bus4,bus5')
  env = simpy.Environment()
  bus1 = simpy.Resource(env,capacity=CAPACIDAD_BUS)
  bus2 = simpy.Resource(env,capacity=CAPACIDAD_BUS)
  bus3 = simpy.Resource(env,capacity=CAPACIDAD_BUS)
  bus4 = simpy.Resource(env,capacity=CAPACIDAD_BUS)
  bus5 = simpy.Resource(env,capacity=CAPACIDAD_BUS)  
  empresa=Empresa(bus1,bus2,bus3,bus4,bus5)  

  def definir_cliente(self):
    p =  0.6
    bernoulli = stats.bernoulli(p)
    genero = bernoulli.rvs(1)
    mu = 35 
    poisson = stats.poisson(mu)  
    edad = poisson.rvs(1)
    origen = random.choices(self.LUGARES, self.PROB_LUGARES, k=1)
    destino = random.choices(self.LUGARES, self.PROB_LUGARES, k=1)  
    tickets=random.randint(1, 6)
    cliente=self.Cliente(genero,edad,origen,destino,tickets)
    return cliente

  def ventaBoletos(self,cliente,bus):
    with bus.request() as turno:
      yield turno
      mu, sigma = 10, 2
      tiempo = np.random.normal(mu, sigma, 1)
      yield self.env.timeout(tiempo)

  def salida_bus(self, proceso):
    yield self.env.timeout(1)
    proceso.interrupt()
    

  def llegadaClientes(self,empresa):  
    indice=0
    while True:
      yield self.env.timeout(self.VELOCIDAD_LLEGADA)
      cliente=self.definir_cliente()
      if(indice==0):
        try:
          self.salida1=self.env.process(self.ventaBoletos(cliente, empresa.bus1))        
        except simpy.Interrupt:
          mu, sigma = 6, 4
          viaje = np.random.normal(mu, sigma, 1)
          yield self.env.timeout(viaje)
          yield self.env.timeout(viaje)        
      if(indice==1):
        try:
          self.salida2=self.env.process(self.ventaBoletos(cliente, empresa.bus2))        
        except simpy.Interrupt:
          mu, sigma = 6, 4
          viaje = np.random.normal(mu, sigma, 1)
          yield self.env.timeout(viaje)
          yield self.env.timeout(viaje)
      if(indice==2):
        try:
          self.salida3=self.env.process(self.ventaBoletos(cliente, empresa.bus3))
        except simpy.Interrupt:
          mu, sigma = 6, 4
          viaje = np.random.normal(mu, sigma, 1)
          yield self.env.timeout(viaje)
          yield self.env.timeout(viaje)
      if(indice==3):
        try:
          self.salida4=self.env.process(self.ventaBoletos(cliente, empresa.bus4))
        except simpy.Interrupt:
          mu, sigma = 6, 4
          viaje = np.random.normal(mu, sigma, 1)
          yield self.env.timeout(viaje)
          yield self.env.timeout(viaje)
      if(indice==4):
        try:
          self.salida5=self.env.process(self.ventaBoletos(cliente, empresa.bus5))
        except simpy.Interrupt:
          mu, sigma = 6, 4
          viaje = np.random.normal(mu, sigma, 1)
          yield self.env.timeout(viaje)
          yield self.env.timeout(viaje)
      indice+=1
      if(indice==5):
        indice=0

  def __init__(self):
    self.env.process(self.llegadaClientes(self.empresa))    
    self.env.run(until=self.TIEMPO_SIMULACION)
simulacion=Simulacion()